### retrieve libraries & authenticate

#### libraries & top vars

In [1]:
import spotipy
import os
import config
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2
import random
import string

inBetaID = config.beta_id

# the queue playlist
# 9/10/2023: this has been reqplaced with the playlist id of the queued playlist 
# also, for future reference, the correct id that should be parsed out of the shared playlist link is the set of numbers right after the playlist/ in the url
pq_id = config.pq_id



#### auth

In [4]:
#todo: don't hard code stuff like this
clientID = 'e7392e632c3745ea9d29acd036bf03f9'
clientSecret = 'bb0d7d5581da42f98ecc610c82803782'
redirectURI = 'http://localhost:8889/callback'
user_id = config.user_id
scopes = [
    'playlist-read-private',
    'playlist-read-collaborative',
    'playlist-modify-private',
    'playlist-modify-public',
    'user-follow-modify',
    'user-follow-read',
    'user-read-playback-position',
    'user-top-read',
    'user-read-recently-played',
    'user-library-modify',
    'user-library-read',
]

scope = ' '.join(scopes)
token = util.prompt_for_user_token(
    username='Larralapid@gmail.com',
    scope=scope,
    client_id=clientID,
    client_secret=clientSecret,
    redirect_uri=redirectURI
)

# create a Spotify object
sp = spotipy.Spotify(auth=token)

#### functions

In [5]:
def get_liked_songs(sp):
    liked_songs_ids = []    
    liked_songs = sp.current_user_saved_tracks()
    
    #keep getting liked songs until we have all of them and update the offset parameter
    while liked_songs['next']:
        liked_songs = sp.current_user_saved_tracks(offset=len(liked_songs_ids))
        for item in liked_songs['items']:
            track = item['track']
            liked_songs_ids.append(track['id'])

    return liked_songs_ids
def get_in_beta(sp,inBetaID):
    in_beta_ids = []
    # get tracks for playlist id 37PJwtSi9QJ6DDT5XnEPXZ
    in_beta = sp.playlist_items(inBetaID)
    
    #keep getting in beta songs until we have all of them and update the offset parameter
    while in_beta['next']:
        #TODO: check if the item is still valid in spotify
        in_beta = sp.playlist_items(inBetaID, offset=len(in_beta_ids))
        for item in in_beta['items']:
            if str(item) == "None" or str(item) == "" or item == None:
                # remove the track from the in_beta playlist
                sp.playlist_remove_all_occurrences_of_items(playlist_id=inBetaID, items=[item])
                print("Removed None or empty string from In Beta")
            else:
                track = item['track']
                in_beta_ids.append(track['id'])
    return in_beta_ids
def get_inbeta_inliked(sp,liked_song_ids,in_beta_ids):
    print('LIKED SONGS: '+ str(liked_song_ids))
    liked_song_ids = set(liked_song_ids)
    # find all tracks in in_beta that are also in liked_songs and add them to a list
    in_beta_and_liked = []
    for track_id in in_beta_ids:
        if track_id in liked_songs_ids:
            in_beta_and_liked.append(track_id)

    return in_beta_and_liked
def create_new_playlist(sp, playlist_name):
    user_id = sp.me()['id']
    playlist = sp.user_playlist_create(user=str(user_id), name=playlist_name, public=False)
    return playlist['id']
def remove_cross_dupes(sp,in_beta_and_liked,inBetaID):
    # remove all tracks in in_beta_and_liked from "in beta" playlist
    items_to_remove = in_beta_and_liked.copy()
    removed_items = []
    while items_to_remove:
        removed = sp.playlist_remove_all_occurrences_of_items(playlist_id=inBetaID, items=items_to_remove[:100])
        removed_items.extend(removed['snapshot_id'])
        items_to_remove = items_to_remove[100:]

    # print the updated total number of tracks in the "in beta" playlist
    in_beta = sp.playlist_items(inBetaID)
    
    return removed_items
def beta_removals(sp, beta_chosen, inBetaID):
    print('INITIATING BETA PLAYLIST REMOVALS')
    # print the count of tracks in the "in beta" playlist
    in_beta = sp.playlist_items(inBetaID)
    print("Total In Beta Before removals:", in_beta['total'])

    # for every item in beta_chosen, remove it from the "in beta" playlist
    for item in beta_chosen:
        sp.playlist_remove_all_occurrences_of_items(playlist_id=inBetaID, items=[item])

    # print the updated count of tracks in the "in beta" playlist
    in_beta = sp.playlist_items(inBetaID)
    print("Total In Beta after removals:", in_beta['total'])
    print('BETA PLAYLIST REMOVALS COMPLETE')
    return in_beta['total']
def queue_it(sp,pq_id,inBetaID):
    # shuffle the liked_songs_ids list
    liked_songs_ids = get_liked_songs(sp)
    random.shuffle(liked_songs_ids)

    new_queue = []
    beta_chosen = get_in_beta(sp,inBetaID)
    beta_chosen = [track for track in beta_chosen if track is not None]

    #for each item in shuffled_liked, add it to the new_queue list and choose a random track from in_beta_ids to add to the new_queue and beta_chosen lists, then remove that track from in_beta_ids
    for item in liked_songs_ids[:]:
        if str(item) == "None" or str(item) == "":
            continue
        else:
            new_queue.append("spotify:track:"+item)
            beta_track = random.choice(beta_chosen)
            new_queue.append("spotify:track:"+beta_track)
            beta_chosen.remove(beta_track)

    # for each item in new_queue_uris, add it to the playlist
    for item in new_queue:
        sp.playlist_add_items(playlist_id=pq_id, items=[item])
    print('QUEUE PLAYLIST POPULATED')

    return beta_removals(sp,beta_chosen,inBetaID)


In [ ]:
# for every track that is in pq_id playlist and also in inBetaID, remove from in beta playlist 
def removeTracksFromInBetaPlaylist(sp,pq_id, inBetaID):
 # Fetch all tracks from inBetaID playlist
    inBeta_track_ids = set()
    offset = 0
    while True:
        inBeta_tracks = sp.playlist_items(inBetaID, offset=offset)
        inBeta_track_ids.update({track['track']['id'] for track in inBeta_tracks['items']})
        
        if not inBeta_tracks['next']:
            break
        offset += len(inBeta_tracks['items'])

    print("count of in beta playlist tracks: ", len(inBeta_track_ids), "\n")
    
    # Check tracks from pq_id against inBeta_track_ids
    to_remove = []
    offset = 0
    while True:
        pq_tracks = sp.playlist_items(pq_id, offset=offset)
        
        for item in pq_tracks['items']:
            if item['track']['id'] in inBeta_track_ids:
                to_remove.append(item['track']['id'])

        if not pq_tracks['next']:
            break
        offset += len(pq_tracks['items'])

    # Batch remove tracks from inBetaID playlist
    for i in range(0, len(to_remove), 100):
        sp.playlist_remove_all_occurrences_of_items(inBetaID, to_remove[i:i+100])
    
    return print("removed ", len(to_remove), " tracks from in beta playlist")




In [ ]:
removeTracksFromInBetaPlaylist(sp,pq_id,inBetaID)

### exe

#### mandatory executions 

In [6]:
queue_it(sp,pq_id,inBetaID)

QUEUE PLAYLIST POPULATED
INITIATING BETA PLAYLIST REMOVALS
Total In Beta Before removals: 4735
Total In Beta after removals: 1690
BETA PLAYLIST REMOVALS COMPLETE


1690

#### optional executions 

In [ ]:
liked_songs_ids = get_liked_songs(sp)
print(liked_songs_ids)

In [ ]:
in_beta_ids = get_in_beta(sp,inBetaID)
print(in_beta_ids)

In [ ]:
in_beta_and_liked = get_inbeta_inliked(sp,liked_songs_ids,in_beta_ids)
print(in_beta_and_liked)

In [ ]:
# remove the tracks that exist in beta and in liked 
remove_cross_dupes(sp,in_beta_and_liked,inBetaID)